In [2]:
from IPython.core.magic import register_line_cell_magic
import subprocess
import sys

In [3]:
@register_line_cell_magic
def nim(line, cell):
    fp = open("for_nim_magic.nim", "w")
    fp.write(cell)
    fp.close()
    p = subprocess.Popen(["nim", "compile", "--run", "for_nim_magic.nim"], stdout=subprocess.PIPE)
    out, err = p.communicate()
    if err:
        print(err.decode("utf-8"), file=sys.stderr)
    if out:
        print(out.decode("utf-8"),"\n\n\n")    

In [53]:
# IF THE URLS need "-d:ssl"

@register_line_cell_magic
def nim(line, cell):
    fp = open("for_nim_magic.nim", "w")
    fp.write(cell)
    fp.close()
    p = subprocess.Popen(["nim", "compile", "--run", "-d:ssl", "for_nim_magic.nim"], stdout=subprocess.PIPE)
    out, err = p.communicate()
    if err:
        print(err.decode("utf-8"), file=sys.stderr)
    if out:
        print(out.decode("utf-8"),"\n\n\n")    

# IMPORTANT!
In the robots.txt file of "https://atlassoftwaredocs.web.cern.ch" it says "Sitemap: /sitemap.xml"

Thus, we can navigate to "https://atlassoftwaredocs.web.cern.ch/sitemap.xml" to simply get all the urls 
that will populate our robots.txt file according to the documentation on this domain!

In [88]:
%%nim
import std/strformat
import std/httpclient
import strutils
import std/os
import std/random

# import std/rdstdin

# TODO
# - process the html
# - get all the data


### FOR ASYNC REQUESTS USE ###
# Async is used when program speed is I/O bound rather than cpu bound
# import std/[asyncdispatch, httpclient]

# proc asyncProc(): Future[string] {.async.} =
#   var client = newAsyncHttpClient()
#   return await client.getContent("http://example.com")

# echo waitFor asyncProc()

##############################

proc return_request*(url: string, idx: int) = 
    
    # runnableExamples:
        # my_request = "http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=10"
        # echo return_request(my_request)

    var 
        client = newHttpClient()
        # Add the request here in this line
        d = client.request(url)
        data = d.body()

    writeFile(&"/home/wrkn/GitRepos/HEPgpt/data-gathering/data/{url.replace('/','.')}_unparsed_{idx}.txt",data)


    # PROCESS IT LATER,htmlparser? ######################3
    # Line by line, we read the file by
    # let f = open(&"request_results_{idx}.txt")

    # var line: string
    # try:
    #     var in_line: bool = false
    #     # while not endOfFile(f):
    #         # line = f.readLine()
    #     for line in lines(f):

    #         if "<title>" in line:
    #             in_line = true
    #             echo line.replace("<title>","Title:")
    #         if "<summary>" in line:
    #             in_line = true
    #             echo line.replace("<summary>","Summary:")

    #         while in_line:
    #             line = f.readLine()

    #             if "</summary>" in line:
    #                 in_line = false
    #                 echo line.replace("</summary>","\n\n")
    #             elif "</title>" in line:
    #                 in_line = false
    #                 echo line.replace("</title>","\n")
    #             else:
    #                 echo line
    # finally:
    #     f.close()


if isMainModule:

    # Define consts and vars.
    # 20,000 and 10,000 means the requests will randomly vary in milliseconds between at least 20s and 20+10=30s
    const minRandDelayInMilliseconds = 20000
    const additionalDelayInMilliseconds = 10000
    var urlList: seq[string]

    # For debugging.
    var randDelay: int


    # Process the urlList from urlFile.txt
    try:
        for line in lines "/home/wrkn/GitRepos/HEPgpt/data-gathering/urlFile.txt":
            urlList.add(line)
    except Exception:
        echo getCurrentExceptionMsg()


    # Put some space between the compiler line and first output line
    echo ""

    # Loop through the urls and return the request in files based on the id of the url.
    # Also, implement a random request delay timer so we wait a minimum and max number of milliseconds between requests according to the robots.txt file (cern.astlas=20000ms min)
    for idx, url in urlList:
        
        try:
            return_request(url, idx)
            echo &"Request {idx} completed.\n"
        except:
            echo "Request failed with error:"
            echo getCurrentExceptionMsg()
        
        # Do not execute sleep on the final iteration
        if idx != len(urlList)-1:
            # For debugging
            randDelay = rand(additionalDelayInMilliseconds) + minRandDelayInMilliseconds
            echo &"randDelay={randDelay}ms"
            os.sleep(randDelay)

            # Non debugging
            # os.sleep(rand(additionalDelayInMilliseconds) + minRandDelayInMilliseconds)

    echo "All procesing complete."

Hint: used config file '/home/wrkn/miniconda3/envs/hepgpt/nim/config/nim.cfg' [Conf]
Hint: used config file '/home/wrkn/miniconda3/envs/hepgpt/nim/config/config.nims' [Conf]
...............................................
Hint:  [Link]
Hint: 99351 lines; 3.611s; 116.348MiB peakmem; Debug build; proj: /home/wrkn/GitRepos/HEPgpt/for_nim_magic.nim; out: /home/wrkn/GitRepos/HEPgpt/for_nim_magic [SuccessX]
Hint: /home/wrkn/GitRepos/HEPgpt/for_nim_magic  [Exec]


CC: stdlib_io.nim
CC: stdlib_system.nim
CC: for_nim_magic.nim

Request 0 completed.

randDelay=20451ms
Request 1 completed.

randDelay=25098ms
All procesing complete.
 



